In [ ]:
#No78(GPU上での活躍)
import torch
#時間の計測
import time
#データセットとデータローダ(データセットの中身をミニバッチごとに固めた集合)の作成に必要
from torch.utils.data import TensorDataset, DataLoader
#プログレスバーの表示
from tqdm import tqdm
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
torch.manual_seed(0)
#モデルの作成
class LogisticRegression(torch.nn.Module):
  #torch.nn.Moduleのサブクラス化(クラスの継承)
    def __init__(self):
        super().__init__()
        self.net = torch.nn.Sequential(torch.nn.Linear(300, 4),)
    def forward(self, X):
        return self.net(X)

model = LogisticRegression()
X_train = torch.load("/content/drive/MyDrive/X_train.pt")
y_train = torch.load("/content/drive/MyDrive/y_train.pt")
X_valid = torch.load("/content/drive/MyDrive/X_valid.pt")
y_valid = torch.load("/content/drive/MyDrive/y_valid.pt")
#GPUを使うようにする
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#今までの結果を踏まえて学習率を0.01にする
optimizer = torch.optim.SGD(model.net.parameters(), lr=0.01)
#損失関数の定義
loss_fn = torch.nn.CrossEntropyLoss()
#batchsizeを２乗していく
batchsize = [2**i for i in range(5)]
#TensorDatasetを作成
ds = TensorDataset(X_train, y_train)
times = []
for bs in batchsize:
    #DataLoaderを作成
    loader = DataLoader(ds, batch_size=bs, shuffle=True)
    train_losses = []
    valid_losses = []
    train_accs = []
    valid_accs = []

    for epoch in tqdm(range(10)):
      #時間の計測をスタートする
        start = time.time()
        for xx, yy in loader:
          #勾配の初期化
            optimizer.zero_grad()
            y_pred = model(X_train)
          #パラメータの更新
            loss = loss_fn(y_pred, y_train)
          #勾配を計算(逆伝播)
            loss.backward()
          #パラメータの更新
            optimizer.step()

        #損失の記録
        train_losses.append(loss.detach().numpy())
        valid_losses.append(loss_fn(model(X_valid), y_valid).detach().numpy())
        #カテゴリの予測(指定した次元の最大値を返す)
        y_max_train, y_pred_train = torch.max(model(X_train),dim=1)
        y_max_valid, y_pred_valid = torch.max(model(X_valid),dim=1)
        #正解率の記録
        train_acc = accuracy_score(y_pred_train, y_train)
        valid_acc = accuracy_score(y_pred_valid, y_valid)
        train_accs.append(train_acc)
        valid_accs.append(valid_acc)

        #学習時間計測
        times.append(time.time() - start)

#計測時間を出力
with open ("/content/drive/MyDrive/knock78output.txt", "w") as f:
    for i in range(len(times)):
      #１０回ごとにバッチサイズを表示
        if i%10 == 0:
            print(f"---------bachsize={2**(i/10)}----------", file=f)
        #iは0から4をとる。余りを利用して、ephck1～10を表記する。
        print(f"epoch{i%10+1}:{times[i]}", file=f)